In [ ]:
import sys
sys.path.append('/home/advice/notebook/jms/우리은행/pytorch-pretrained-BERT/examples/')
sys.path.append('/home/advice/notebook/jms/우리은행/pytorch-pretrained-BERT/')

In [ ]:
from run_squad_spm import *
from  pytorch_pretrained_bert import modeling
import torch
import torch.nn.functional as F

In [ ]:
from datetime import date
today = date.today()
today = str(today).replace('-', '')
direc = '/home/advice/notebook/jms/우리은행/output_dir/'
try:
    os.mkdir(direc+today)
    print(direc , today ,  " Created ") 
except FileExistsError:
    print(direc , today ,  " already exists")

In [ ]:
import csv
from pathlib import Path
import torch
import re
from torch import Tensor, nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss

import pandas as pd
import collections
import os
import pdb
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from sklearn.metrics import roc_curve, auc
from tqdm import tqdm_notebook as tqdm

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pytorch_pretrained_bert.optimization import BertAdam

In [ ]:
from torch.optim import Adam

## set hyper params

In [ ]:
model_dir = "/home/advice/notebook/jms/우리은행/extract_kobert/"

vocab_file = '/home/advice/notebook/jms/kobert/kobert_news_wiki_ko_cased-1087f8699e.spiece'
bert_config_file = model_dir + "kobert_config.json"
init_checkpoint = model_dir + "kobert_model.bin"
bert_model = 'kobert'
PATH = "/home/advice/notebook/jms/우리은행/data/"

output_dir = direc+today+'/'
train_file = "news_tr_small_5.txt"
eval_file = "news_te.txt"


In [ ]:
args = {
    "train_size": -1,
    "val_size": -1,
    "train_file": train_file,
    "eval_file": eval_file,
    "data_dir": PATH,
    "task_name": "news_multilabel",
    "no_cuda": False,
    "bert_model": model_dir,
    "output_dir": output_dir,
    "tokenizer": vocab_file,
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 32,
    "eval_batch_size": 32,
    "learning_rate": 3e-5,
    "num_train_epochs": 4.0,
    "warmup_proportion": 0.1,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": False,
    "loss_scale": 128
}

In [ ]:
tokenizer = BERTSPMTokenizer.from_pretrained(args['tokenizer'])

In [ ]:
with open('/home/advice/notebook/jms/우리은행/data/news_tr_small_1.txt', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    lines = []
    for line in reader:
        lines.append(line)
tokenizer.tokenize(lines[3][0])

In [ ]:
from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification

In [ ]:
class AvgVec(nn.Module):
    def __init__(self):
        super(AvgVec, self).__init__()

    def forward(self, hidden_states, mask):
        last_layer = hidden_states[-1]
        input_vecs = last_layer

        sum_vecs = (input_vecs * mask.unsqueeze(-1)).sum(1)
        avg_vecs = sum_vecs / mask.sum(1, keepdim=True)
        return avg_vecs


class BertForSiameseClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels=2):
        super(BertForSiameseClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, 2)
        self.apply(self.init_bert_weights)
        self.avg_vec = AvgVec()

    def forward(self, input_ids_1, input_mask_1, input_ids_2, input_mask_2):
        self.bert.eval()
        encoder_layer_1, pooled_output_1 = self.bert(input_ids_1, token_type_ids=None, attention_mask=input_mask_1)
        encoder_layer_2, pooled_output_2 = self.bert(input_ids_2, token_type_ids=None, attention_mask=input_mask_2)
        out1 = self.avg_vec(encoder_layer_1, input_mask_1)
        out2 = self.avg_vec(encoder_layer_2, input_mask_2)
        out_norm = diff = torch.abs(out1 - out2)
        logit = self.classifier(out_norm)
        softmax = F.softmax(logit, dim=1)
        return logit, softmax

In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        


In [ ]:
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()
    
    def get_test_examples(self, data_dir, data_file_name, size=-1):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError() 

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

In [ ]:
with open('/home/advice/notebook/jms/우리은행/data/news_tr.txt', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    lines = []
    for line in reader:
        lines.append(line)

class MultiLabelTextProcessor(DataProcessor):

    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.labels = None

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines    
    
    def get_train_examples(self, data_dir, size=-1):
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, args['train_file'])))
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, args['train_file'])), "train")
        
    def get_dev_examples(self, data_dir, size=-1):
        filename = 'news_te.txt'
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, args['eval_file'])))
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, filename)), "dev")
    
    def get_test_examples(self, data_dir, data_file_name, size=-1):
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, args['eval_file'])))
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, args['eval_file'])), "test")

    def get_labels(self):
        """See base class."""
        if self.labels == None:
            self.labels = ['0', '1', '2', '3', '4', '5']
        return self.labels

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            text_b = None
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=text_b, labels=label))
        return examples

In [ ]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(float(label))

#         label_id = label_map[example.label]
        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features


In [ ]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [ ]:
def accuracy(out, labels):
    out_cpu = out.cpu().numpy()
    labels_cpu = labels.cpu().numpy()
    outputs = np.argmax(out_cpu, axis=1)
    return np.sum(outputs == labels_cpu)

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: y_pred = y_pred.sigmoid()
#     return ((y_pred>thresh)==y_true.byte()).float().mean().item()
    return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.2, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

In [ ]:
processors = {
    "news_multilabel": MultiLabelTextProcessor
}

# Setup GPU parameters

if args["local_rank"] == -1 or args["no_cuda"]:
    device = torch.device("cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu")
    n_gpu = torch.cuda.device_count()
#     n_gpu = 1
else:
    torch.cuda.set_device(args['local_rank'])
    device = torch.device("cuda", args['local_rank'])
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args['local_rank'] != -1), args['fp16']))

In [ ]:
random.seed(args['seed'])
np.random.seed(args['seed'])
torch.manual_seed(args['seed'])
if n_gpu > 0:
    torch.cuda.manual_seed_all(args['seed'])

In [ ]:
task_name = args['task_name'].lower()
if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))
    
processor = processors[task_name](args['data_dir'])
label_list = processor.get_labels()
num_labels = len(label_list)

In [ ]:
tokenizer = BERTSPMTokenizer.from_pretrained(args['tokenizer'])
train_examples = None
num_train_steps = None
if args['do_train']:
    train_examples = processor.get_train_examples(args['data_dir'], size=args['train_size'])
#     train_examples = processor.get_train_examples(args['data_dir'], size=args['train_size'])
    num_train_steps = int(
        len(train_examples) / args['train_batch_size'] / args['gradient_accumulation_steps'] * args['num_train_epochs'])

In [ ]:
train_features = convert_examples_to_features(
    train_examples, label_list, args['max_seq_length'], tokenizer)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() and not args["no_cuda"] else "cpu")
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
model = BertForSiameseClassification(bert_config, num_labels = num_labels)
model.bert.load_state_dict(torch.load(init_checkpoint))
model = torch.nn.DataParallel(model)
model.to(device)

In [ ]:
class SiameseFeatures(object):
    """A single set of features of data."""
    def __init__(self,feature_id, data_a, data_a_mask, data_a_lab,
                 data_b, data_b_mask, data_b_lab, label):
        self.feature_id = feature_id
        self.data_a = data_a
        self.data_a_mask = data_a_mask
        self.data_a_lab = data_a_lab
        self.data_b = data_b
        self.data_b_mask = data_b_mask
        self.data_b_lab = data_b_lab
        self.label = label

def convert_to_siamese_features(tf):
    siamese_features = []
    tf_len = len(tf)
    for idx in range(tf_len):
        data_a  = tf[idx].input_ids
        data_a_mask = tf[idx].input_mask
        data_a_lab = tf[idx].label_ids
        for idx_2 in range(idx, tf_len):
            data_b = tf[idx_2].input_ids
            data_b_mask = tf[idx_2].input_mask
            data_b_lab = tf[idx_2].label_ids
            label = int(np.where(tf[idx].label_ids == tf[idx_2].label_ids, 
                             1,0))
            siamese_features.append(SiameseFeatures(feature_id = 'train_{}_{}'.format(int(idx), int(idx_2)), 
                                                    data_a = data_a, 
                                                    data_a_mask = data_a_mask, 
                                                    data_a_lab = data_a_lab, 
                                                    data_b = data_b, 
                                                    data_b_mask = data_b_mask, 
                                                    data_b_lab = data_b_lab, 
                                                    label = label))
    return siamese_features

In [ ]:
siamese_features = convert_to_siamese_features(train_features)

In [ ]:
all_data_a = torch.tensor([f.data_a for f in siamese_features],dtype=torch.long).to(device)
all_data_a_mask = torch.tensor([f.data_a_mask for f in siamese_features],dtype=torch.float).to(device)
all_data_b = torch.tensor([f.data_b for f in siamese_features],dtype=torch.long).to(device)
all_data_b_mask = torch.tensor([f.data_b_mask for f in siamese_features],dtype=torch.float).to(device)
all_label = torch.tensor([f.label for f in siamese_features]).to(device)

In [ ]:
all_data_a.shape

In [ ]:
train_data = TensorDataset(all_data_a, all_data_a_mask, all_data_b, all_data_b_mask, all_label)

In [ ]:
train_sampler = RandomSampler(train_data)

In [ ]:
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=4)

In [ ]:
iter=10
model.train()

optimizer = Adam(model.module.classifier.parameters(), lr=0.1)

loss_total = 0.0
trial = 0
for _ in range(iter) : 
    for step, batch in enumerate(train_dataloader):
        dat_a, mask_a, dat_b, mask_b, lab= batch
        logit, _ = model(dat_a, mask_a, dat_b, mask_b)
        loss_func = torch.nn.CrossEntropyLoss() 
        loss = loss_func(logit, lab)
        if n_gpu>1:
            loss = loss.mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_total += loss.item()
        trial += 1

    print("loss : ", loss_total/trial)


In [ ]:
predict_processor = MultiLabelTextProcessor(PATH)
test_examples = predict_processor.get_test_examples(PATH, 'news_te.csv', size=-1)

In [ ]:
test_features = convert_examples_to_features(
    test_examples, label_list, args['max_seq_length'], tokenizer)

In [ ]:
def convert_to_siamese_features_for_test(test, train):
    siamese_features = []
    test_len = len(test)
    train_len = len(train)
    for idx in range(test_len):
        data_a  = test[idx].input_ids
        data_a_mask = test[idx].input_mask
        data_a_lab = test[idx].label_ids
        for idx_2 in range(train_len):
            data_b = train[idx_2].input_ids
            data_b_mask = train[idx_2].input_mask
            data_b_lab = train[idx_2].label_ids
            label = int(np.where(test[idx].label_ids == train[idx_2].label_ids, 
                             1,0))
            siamese_features.append(SiameseFeatures(feature_id = 'test_{}'.format(int(idx)), 
                                                    data_a = data_a, 
                                                    data_a_mask = data_a_mask, 
                                                    data_a_lab = data_a_lab, 
                                                    data_b = data_b, 
                                                    data_b_mask = data_b_mask, 
                                                    data_b_lab = data_b_lab, 
                                                    label = label))
    return siamese_features

In [ ]:
test_set = convert_to_siamese_features_for_test(test_features[:100], train_features)

In [ ]:
len(test_set)

In [ ]:
all_test_data = torch.tensor([f.data_a for f in test_set],dtype=torch.long).to(device)
all_test_data_mask = torch.tensor([f.data_a_mask for f in test_set],dtype=torch.float).to(device)
all_compare_data = torch.tensor([f.data_b for f in test_set],dtype=torch.long).to(device)
all_compare_data_mask = torch.tensor([f.data_b_mask for f in test_set],dtype=torch.float).to(device)
test_ans = [f.data_a_lab[0] for f in test_set]
test_id = [f.feature_id for f in test_set]
train_lab = [f.data_b_lab[0] for f in test_set]

In [ ]:
train_lab = [f.data_b_lab[0] for f in test_set]

In [ ]:
test_data = TensorDataset(all_test_data, all_test_data_mask, all_compare_data, all_compare_data_mask)

In [ ]:
test_sampler = SequentialSampler(test_data)

In [ ]:
eval_batch_size = 32

In [ ]:
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=eval_batch_size)

In [ ]:
model.eval()
all_logits = None
for step, batch in enumerate(tqdm(test_dataloader, desc="Prediction Iteration")):
            test_data, test_data_mask, compare_data, compare_data_mask = batch
            test_data = test_data.to(device)
            test_data_mask = test_data_mask.to(device)
            compare_data = compare_data.to(device)
            compare_data_mask = compare_data_mask.to(device)
            with torch.no_grad():
                _,logits = model(test_data, test_data_mask, compare_data, compare_data_mask)
                if all_logits is None:
                    all_logits = logits.detach().cpu().numpy()
                else:
                    all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
           

In [ ]:
result = pd.DataFrame({'test_id': test_id, 'train_label':train_lab, 'prob':all_logits[:,1],'real':test_ans})

In [ ]:
result.head(10)

In [ ]:
result_pivot = pd.pivot_table(result, values='prob', 
                              index = ['test_id', 'real'], 
                              columns = ['train_label'], aggfunc = np.mean)

In [ ]:
result_pivot

In [ ]:
result_pivot.loc[:,'pred'] = result_pivot.apply(lambda row: np.argmax(row), axis = 1)
result_pivot = result_pivot.reset_index()
result_pivot[result_pivot['real'] == result_pivot['pred']].shape[0]/result_pivot.shape[0]

In [ ]:
result_pivot = result_pivot.reset_index()
result_pivot[result_pivot['real'] == result_pivot['pred']].shape[0]/result_pivot.shape[0]

In [ ]:
result_pivot[result_pivot['real'] == result_pivot['pred']].shape[0]/result_pivot.shape[0]

In [ ]:
result_pivot[result_pivot['real'] == result_pivot['pred']].shape[0]/result_pivot.shape[0]

In [ ]:
all_input_ids = torch.tensor([[f.input_ids] for f in train_features],dtype=torch.long).to(device)
all_input_mask = torch.tensor([[f.input_mask] for f in train_features],dtype=torch.float).to(device)
all_label_ids = torch.tensor([f.label_ids for f in train_features]).to(device)

In [ ]:
# all_input_ids = [f.input_ids for f in train_features]
# all_input_mask = [f.input_mask for f in train_features]
# all_label_ids = [f.label_ids for f in train_features]

In [ ]:
data_set = []
for idx in range(len(train_features)):
    tmp = [[[all_input_ids[idx], all_input_mask[idx], all_label_ids[idx]],
            [all_input_ids[idx_2], all_input_mask[idx_2], all_label_ids[idx_2]], 
            int(np.where(idx==idx_2, 1,0))] for idx_2 in range(idx,len(train_features)) ]
    data_set.extend(tmp)

In [ ]:
train_data = TensorDataset(data_set)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=1)

In [ ]:
data_set

In [ ]:
a1 = torch.tensor([i[0][0]for i in data_set],dtype=torch.long).to(device)
a2 = torch.tensor([i[0][1]for i in data_set],dtype=torch.float).to(device)
b1 = torch.tensor([i[1][0]for i in data_set],dtype=torch.long).to(device)
b2 = torch.tensor([i[1][1]for i in data_set],dtype=torch.float).to(device)
lab = torch.tensor([i[2] for i in data_set]).to(device)

In [ ]:
# train_data = TensorDataset(a1, a2, b1, b2,lab)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=1)

In [ ]:
# batch_size=1
# iter=10
# model.train()

# optimizer = Adam(model.module.classifier.parameters(), lr=0.1)

# loss_total = 0.0
# trial = 0
# for _ in range(iter) : 
#     print(0)
#     for step, batch in enumerate(train_dataloader):
#         print(1)
#         a1,a2,b1,b2,lab= batch
#         print(2)
#         logit, _ = model(a1,a2,b1,b2)
#         print(3)
#         loss_func = torch.nn.CrossEntropyLoss() 
#         loss = loss_func(logit, lab)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         loss_total += loss.item()
#         trial += 1

#     print("loss : ", loss_total/trial)


In [ ]:
batch_size=1
iter=4
model.cuda()
model.train()

optimizer = Adam(model.module.classifier.parameters(), lr=0.1)

loss_total = 0.0
trial = 0
for _ in range(iter) : 
    for data in data_set:
        logit, _ = model(data[0][0], data[0][1], data[1][0], data[1][1])
        y  = torch.tensor([data[2]]).cuda()
        loss_func = torch.nn.CrossEntropyLoss() 
        loss = loss_func(logit, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_total += loss.item()
        trial += 1

    print("loss : ", loss_total/trial)


In [ ]:
predict_processor = MultiLabelTextProcessor(PATH)
test_examples = predict_processor.get_test_examples(PATH, 'news_te.csv', size=-1)

# Hold input data for returning it 
input_data = [{ 'id': input_example.guid, 'comment_text': input_example.text_a } for input_example in test_examples]

test_features = convert_examples_to_features(
    test_examples, label_list, args['max_seq_length'], tokenizer)


all_input_ids_test = torch.tensor([[f.input_ids] for f in test_features],dtype=torch.long).to(device)
all_input_mask_test = torch.tensor([[f.input_mask] for f in test_features],dtype=torch.float).to(device)
ans = [int(f.label_ids[0]) for f in test_features]

In [ ]:
ans[3]

In [ ]:
def test_oneshot(test_id, test_mask) : 
    model.eval()
    prob_list = []
    for idx in range(len(train_features)) :
        p = model(test_id, test_mask, all_input_ids[idx], all_input_mask[idx])
        prob_list.append(p[1][0][1])
#         print("Prob of {0} : {1}".format(idx, p[1][0][1]))
#     print('predict: ',np.argmax(prob_list))
    return np.argmax(prob_list)

In [ ]:
test_oneshot(all_input_ids_test[3], all_input_mask_test[3])

In [ ]:
test_oneshot(all_input_ids_test[10], all_input_mask_test[10])

In [ ]:
ans[10]

In [ ]:
x = []
for i in range(len(test_features)):
    k = ans[i]==test_oneshot(all_input_ids_test[i], all_input_mask_test[i])
    print(ans[i]==test_oneshot(all_input_ids_test[i], all_input_mask_test[i]))
    print('real: ',ans[i])
    x.append(k)

In [ ]:
np.sum(x)/len(x)

In [ ]:
x = []
for i in range(len(test_features)):
    k = ans[i]==test_oneshot(all_input_ids_test[i], all_input_mask_test[i])
    print(ans[i]==test_oneshot(all_input_ids_test[i], all_input_mask_test[i]))
    print('real: ',ans[i])
    x.append(k)

In [ ]:
np.sum(x)

In [ ]:
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", args['train_batch_size'])
logger.info("  Num steps = %d", num_train_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features])#, dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features])#, dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in train_features])#, dtype=torch.long)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
if args['local_rank'] == -1:
    train_sampler = RandomSampler(train_data)
else:
    train_sampler = DistributedSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args['train_batch_size'])

In [ ]:
model.module.unfreeze_bert_encoder()

In [ ]:
num_epocs=args['num_train_epochs']

global_step = 0
model.train()
for i_ in tqdm(range(int(num_epocs)), desc="Epoch"):

    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
        if args['gradient_accumulation_steps'] > 1:
            loss = loss / args['gradient_accumulation_steps']

        if args['fp16']:
            optimizer.backward(loss)
        else:
            loss.backward()

        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % args['gradient_accumulation_steps'] == 0:
#             scheduler.batch_step()
            # modify learning rate with special warm up BERT uses
            lr_this_step = args['learning_rate'] * warmup_linear(global_step/t_total, args['warmup_proportion'])
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

    logger.info('Loss after epoc {}'.format(tr_loss / nb_tr_steps))
    logger.info('Eval after epoc {}'.format(i_+1))
    eval()

In [ ]:
# Save a trained model
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join(output_dir, "finetuned_pytorch_model.bin")
torch.save(model_to_save.state_dict(), output_model_file)

In [ ]:
# Load a trained model that you have fine-tuned
model_state_dict = torch.load(output_model_file)
model = BertForSiameseClassification.from_pretrained(args['bert_model'], num_labels = num_labels, state_dict=model_state_dict)
model.to(device)

In [ ]:
def predict(model, path, test_filename='news_te.csv'):
    predict_processor = MultiLabelTextProcessor(path)
    test_examples = predict_processor.get_test_examples(path, test_filename, size=-1)
    
    # Hold input data for returning it 
    input_data = [{ 'id': input_example.guid, 'comment_text': input_example.text_a } for input_example in test_examples]

    test_features = convert_examples_to_features(
        test_examples, label_list, args['max_seq_length'], tokenizer)
    
    logger.info("***** Running prediction *****")
    logger.info("  Num examples = %d", len(test_examples))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    
    # Run prediction for full data
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=args['eval_batch_size'])
    
    all_logits = None
    
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(tqdm(test_dataloader, desc="Prediction Iteration")):
        input_ids, input_mask, segment_ids = batch
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)
            logits = logits.sigmoid()

        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    return pd.merge(pd.DataFrame(input_data), pd.DataFrame(all_logits, columns=label_list), left_index=True, right_index=True)

In [ ]:
a = predict(model, PATH)

In [ ]:
a

In [ ]:
a.loc[:, 'pred'] = a.iloc[:,2:].apply(lambda x: x.idxmax(), axis = 1)

In [ ]:
with open('/home/advice/notebook/jms/우리은행/data/news_te.txt', "r") as f:
    reader = csv.reader(f, delimiter="\t", quotechar=None)
    lines = []
    for line in reader:
        lines.append(line)
t = [i[1] for i in lines[1:]]
a.loc[:,'real'] = t

In [ ]:
a[a.pred == a.real].shape[0]/a.shape[0]

In [ ]:
a

In [ ]:
def fit(num_epocs=args['num_train_epochs']):
    global_step = 0
    model.train()
    for i_ in tqdm(range(int(num_epocs)), desc="Epoch"):

        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):

            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label_ids = batch
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                optimizer.backward(loss)
            else:
                loss.backward()

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
    #             scheduler.batch_step()
                # modify learning rate with special warm up BERT uses
                lr_this_step = args['learning_rate'] * warmup_linear(global_step/t_total, args['warmup_proportion'])
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_this_step
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

        logger.info('Loss after epoc {}'.format(tr_loss / nb_tr_steps))
        logger.info('Eval after epoc {}'.format(i_+1))
        eval()

In [ ]:
fit()